## Import the required packages

In [1]:
import pandas as pd
import numpy as np

## Read the csv data
#### It is important to resave the `.xlxs ` file into a `.csv` file before getting pandas to read the file.
- keep_default_na is set to False so it will recognise other stuff as na
- if you want to change all the empty spaces to `NaN`, include `na_values=['']`, which tells pandas to recognize empty spaces as `NaN`

In [2]:
df = pd.read_csv('Data/freshie1617.csv',skiprows=[0,1,2],parse_dates='Date of Birth',keep_default_na=False)

 Here we replace the empty spaces with float 10 as empty spaces in the data indicate that the students did not take that particular subject. It makes it easier to classify the students that have not taking some subjects into the bridging groups.

In [3]:
df = df.replace('',float(10))

We now change the grades of the students into float so we can make use of the ordering of the numbers to sort the students.

In [4]:
df[['G42 - Additional Mathematics','G46 - Biology','G51 - Science (Physics, Biology)',
    'G52 - Science (Chemistry, Biology)']] = df[['G42 - Additional Mathematics','G46 - Biology',
    'G51 - Science (Physics, Biology)','G52 - Science (Chemistry, Biology)']].astype(float)

We can use `.dtypes` to check the type of the data for the different columns.

In [5]:
df.dtypes

S/N                                                        int64
Adm No                                                    object
Student Name                                              object
Acad Course Code                                          object
Course Abbr                                               object
Student NRIC/ Passport                                    object
Date of Birth                                             object
Gender                                                    object
Entry Qual Code                                           object
Adm Category                                              object
Personal Email Address                                    object
Is Fitness                                                object
Is C Asthma                                               object
Mobile No.                                                 int64
Home Tel No.                                              object
House Blk                

We can use `.pivot_table` to look at how the different diplomas perform for a particular subject, in this case is additional mathematics. Here, those with grade value of 10 means that the student did not take additional mathematics as we have earlier replaced all the empty spaces with the float value of 10. 

Note: This step can be skipped, it is more like a checking for myself.

In [6]:
df[['Adm No','Course Abbr','G42 - Additional Mathematics']].pivot_table(index='Course Abbr', 
        columns='G42 - Additional Mathematics',aggfunc='count',fill_value=0,margins=True)

Adm No                                       
G42 - Additional Mathematics    1.0 2.0  3.0 4.0 5.0 6.0 8.0 9.0 10.0  All
Course Abbr                                                               
BCS                               0   1    8   2   0   3   0   2   19   35
BIO                               4  16   23   5   0   4   1   1   17   71
BMS                               6   6   13   3   0   2   0   0   11   41
ChE                               0  18   29   9   4   2   1   1   49  113
FSN                               0   3   19   4   2   4   0   0   23   55
PHS                               3  19   23   4   3   2   0   0   15   69
VET                               6  13   16   2   1   0   0   0    6   44
All                              19  76  131  29  10  17   2   4  140  428

#### We can also group the students according to their courses.

In [7]:
che = df[df['Course Abbr']=='ChE']
bio = df[df['Course Abbr']=='BIO']
phs = df[df['Course Abbr']=='PHS']
fsn = df[df['Course Abbr']=='FSN']
vet = df[df['Course Abbr']=='VET']
bms = df[df['Course Abbr']=='BMS']
bcs = df[df['Course Abbr']=='BCS']

Since only ChE students need to take their mathematics, we just pull out ChE freshies that had a grade of 6 or lower for their additional mathematics. See that those students who didn't take additional mathematics have a grade of 10 and they are also pulled out at the same time. If we were to assign the grades for students who didn't take a particular subject as `NaN`, the classification would have been more tedious (I think?).

In [8]:
che_em_list = che[che['G42 - Additional Mathematics']>5]
print 'Number of ChE students to go for math bridging:', len(che_em_list)

Number of ChE students to go for math bridging: 53


As I checked with the Bridging programme chairman that he needed the following information to create the attendance list, I have stored them into the variable `info` so as to reduce the typing to be done.

In [9]:
info = ['Student Name','Adm No','Acad Course Code','Course Abbr','Mobile No.','Personal Email Address']

#### One bridging class down!
We can now save the name and info of ChE students that have to take the math bridging into a csv file named `che_em_list.csv`

In [10]:
che_em_list[info].to_csv('C:/Users/laizs/Desktop/notebook/bridging/che_em_list.csv')

I'll leave the rest as an exercise for the reader to see how the remaining attendance for the different classes are obtained

As a guide, the naming terminologies for the csv files are given below:
- hap_mas_list: students that have to do bridigng for mas and hap
- mas_list: students that only have to do bridging for mas
- hap_list: students that only have to do bridging for hap

In [11]:
mas_pop = df[((df['Course Abbr']=='BMS') | (df['Course Abbr']=='BIO')| (df['Course Abbr']=='FSN')| (df['Course Abbr']=='PHS'))
             &(df['G42 - Additional Mathematics']>5)]
len(mas_pop)

80

In [12]:
mas_pop.to_csv('C:/Users/laizs/Desktop/notebook/bridging/mas_pop.csv')
mas_pop['G52 - Science (Chemistry, Biology)'].unique()

array([ 10.,   3.,   2.,   1.,   5.])

In [13]:
hap_mas_list = mas_pop[(mas_pop['G46 - Biology']>5)&(mas_pop['G51 - Science (Physics, Biology)']>5)&
                     (mas_pop['G52 - Science (Chemistry, Biology)']>5)]

mas_list = mas_pop[(mas_pop['G46 - Biology']<=5)|(mas_pop['G51 - Science (Physics, Biology)']<=5)|
                     (mas_pop['G52 - Science (Chemistry, Biology)']<=5)]


In [14]:
print len(hap_mas_list[info])
print len(mas_list[info])


40
40


In [15]:
hap_mas_list[info].to_csv('C:/Users/laizs/Desktop/notebook/bridging/hap_mas_list.csv')
mas_list[info].to_csv('C:/Users/laizs/Desktop/notebook/bridging/mas_list.csv')


In [16]:
hap_pop = df[((df['Course Abbr']=='BMS') | (df['Course Abbr']=='BIO')| (df['Course Abbr']=='FSN')| (df['Course Abbr']=='PHS'))
             & ~((df['G46 - Biology']<=5)|(df['G51 - Science (Physics, Biology)']<=5)|(df['G52 - Science (Chemistry, Biology)']<=5))]
len(hap_pop)

101

In [17]:
hap_list = hap_pop[hap_pop['G42 - Additional Mathematics']<=5]
len(hap_list)

61

In [18]:
hap_list[info].to_csv('C:/Users/laizs/Desktop/notebook/bridging/hap_list.csv')